In [41]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import IPython 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium


In [3]:
CLIENT_ID = 'RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5' # your Foursquare ID
CLIENT_SECRET = 'TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [4]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent") #creating an instance of the geocoder
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [5]:
search_query = 'Italian'
radius = 500

In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5&client_secret=TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [1]:
results = requests.get(url).json()
results

NameError: name 'requests' is not defined

In [31]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1578013384,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,...,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",294544,https://www.seamless.com/menu/harrys-italian-p...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1578013384,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,...,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1578013384,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,...,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,NaN,NaN,NaN,NaN,NaN,NaN,btwn Church St & W Broadway


In [32]:
dataframe['categories']

0    [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...
1    [{'id': '4d4b7105d754a06374d81259', 'name': 'F...
2    [{'id': '4bf58dd8d48988d110941735', 'name': 'I...
Name: categories, dtype: object

In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


In [34]:
dataframe_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [36]:
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

In [37]:
dataframe_filtered.columns= [x.split('.')[-1] for x in dataframe_filtered.columns]

In [38]:
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [47]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Explore A Given Venue

In [49]:
dataframe_filtered.iloc[0]

name                                        Harry's Italian Pizza Bar
categories                                                Pizza Place
address                                                 225 Murray St
lat                                                           40.7152
lng                                                          -74.0147
labeledLatLngs      [{'label': 'display', 'lat': 40.71521779064671...
distance                                                           58
postalCode                                                      10282
cc                                                                 US
city                                                         New York
state                                                              NY
country                                                 United States
formattedAddress    [225 Murray St, New York, NY 10282, United Sta...
crossStreet                                                       NaN
id                  

In [50]:
venue_id='4fa862b3e4b0ebff2f749f06'

In [51]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5&client_secret=TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB&v=20180604'

In [68]:
result = requests.get(url).json()
result['response']['venue']

KeyError: 'venue'

In [57]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [59]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [60]:
dataframe_filtered.iloc[2]

name                                                             Ecco
categories                                         Italian Restaurant
address                                               124 Chambers St
lat                                                           40.7153
lng                                                          -74.0088
labeledLatLngs      [{'label': 'display', 'lat': 40.71533713859952...
distance                                                          549
postalCode                                                      10007
cc                                                                 US
city                                                         New York
state                                                              NY
country                                                 United States
formattedAddress    [124 Chambers St (btwn Church St & W Broadway)...
crossStreet                               btwn Church St & W Broadway
id                  

In [73]:
venue_id='3fd66200f964a520f4e41ee3'

In [92]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)


In [94]:
result1=requests.get(url).json()
result1['response']['venue']

{'id': '3fd66200f964a520f4e41ee3',
 'name': 'Ecco',
 'contact': {'phone': '2122277074',
  'formattedPhone': '(212) 227-7074',
  'facebook': '496751683729009',
  'facebookUsername': 'EccoRestaurant',
  'facebookName': 'Ecco'},
 'location': {'address': '124 Chambers St',
  'crossStreet': 'btwn Church St & W Broadway',
  'lat': 40.71533713859952,
  'lng': -74.00884766217825,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71533713859952,
    'lng': -74.00884766217825}],
  'postalCode': '10007',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['124 Chambers St (btwn Church St & W Broadway)',
   'New York, NY 10007',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/ecco/3fd66200f964a520f4e41ee3',
 'categories': [{'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restaurants',
   'shortName': 'Italian',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/

In [96]:
print(result1['response']['venue']['rating'])

7.9


In [106]:
print(result1['response']['venue']['tips']['count'])

18


In [108]:
limit=15

In [125]:
url='https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION,LIMIT)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3/tips?client_id=RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5&client_secret=TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB&v=20180604&limit=30'

In [126]:
results1=requests.get(url).json()

In [131]:
tips=results1['response']['tips']['items']

In [138]:
results1['response']['tips']['items'][0]

{'id': '5ab1cb46c9a517174651d3fe',
 'createdAt': 1521601350,
 'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
 'type': 'user',
 'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
 'lang': 'en',
 'likes': {'count': 0, 'groups': []},
 'logView': True,
 'agreeCount': 4,
 'disagreeCount': 0,
 'lastVoteText': 'Upvoted 2 weeks ago',
 'lastUpvoteTimestamp': 1576365433,
 'todo': {'count': 0},
 'user': {'id': '0',
  'firstName': 'Nick',
  'lastName': 'E',
  'gender': 'male',
  'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
   'suffix': '/blank_boy.png'}},
 'authorInteractionType': 'liked'}

In [137]:
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

In [139]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,E,male,0


In [169]:
user_id='12'

In [170]:
url= 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [171]:
requests.get(url).json()

{'meta': {'code': 400,
  'errorType': 'param_error',
  'errorDetail': "Must provide a valid user ID or 'self.'",
  'requestId': '5e0eaf5f02a172001ba2cf77'},
 'response': {}}

In [161]:
user_id = '0' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL



# Explore A Venue


In [172]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5&client_secret=TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB&ll=40.7149555,-74.0153365&v=20180604&radius=500&limit=30'

In [181]:
r=requests.get(url).json()
'There are {} spots around the restaurant'.format(len(r['response']['groups'][0]['items']))

'There are 30 spots around the restaurant'

In [187]:
items=r['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '51bcd723498eecb51dd996bc',
  'name': 'Asphalt Green Battery Park City',
  'location': {'address': '212 N End Ave',
   'crossStreet': 'Murray Street',
   'lat': 40.715923644223366,
   'lng': -74.01467345576101,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.715923644223366,
     'lng': -74.01467345576101}],
   'distance': 121,
   'postalCode': '10282',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['212 N End Ave (Murray Street)',
    'New York, NY 10282',
    'United States']},
  'categories': [{'id': '4f4528bc4b90abdf24c9de85',
    'name': 'Athletics & Sports',
    'pluralName': 'Athletics & Sports',
    'shortName': 'Athletics & Sports',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
     'suffix': '.png'},
   

In [188]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


In [194]:
dataframe_filtered.columns=[x.split('.')[-1] for x in dataframe_filtered.columns ]

In [ ]:
dataframe_filtered['venue.categories']=dataframe_filtered.apply(get_category_type, axis=1)

In [195]:
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Asphalt Green Battery Park City,Athletics & Sports,212 N End Ave,Murray Street,40.715924,-74.014673,"[{'label': 'display', 'lat': 40.715923644223366, 'lng': -74.01467345576101}]",121,10282,US,New York,NY,United States,"[212 N End Ave (Murray Street), New York, NY 10282, United States]",NaN,51bcd723498eecb51dd996bc
1,"Waterfront Plaza, Brookfield Place",Plaza,230 Vesey Street,NaN,40.713241,-74.016241,"[{'label': 'display', 'lat': 40.71324098127375, 'lng': -74.01624118660538}]",205,10281,US,New York,NY,United States,"[230 Vesey Street, New York, NY 10281, United States]",NaN,4a0b04dcf964a520ba741fe3
2,Brookfield Place (BFPL),Shopping Mall,230 Vesey St,at West St,40.713240,-74.015193,"[{'label': 'display', 'lat': 40.71324018931094, 'lng': -74.01519298553465}]",191,10281,US,New York,NY,United States,"[230 Vesey St (at West St), New York, NY 10281, United States]",NaN,4a6fa0abf964a520a4d61fe3
3,Irish Hunger Memorial,Monument / Landmark,Vesey St & North End Ave,NaN,40.715139,-74.016557,"[{'label': 'display', 'lat': 40.71513873128946, 'lng': -74.01655655956712}]",104,10282,US,New York,NY,United States,"[Vesey St & North End Ave, New York, NY 10282, United States]",Battery Park City,4a397f7ef964a520839f1fe3
4,Teardrop Park,Park,Warren St,btwn River Terrace & North End Ave,40.716843,-74.015576,"[{'label': 'display', 'lat': 40.716843120089855, 'lng': -74.01557592134671}]",211,10005,US,New York,NY,United States,"[Warren St (btwn River Terrace & North End Ave), New York, NY 10005, United States]",NaN,4a6757aaf964a52037c91fe3
5,Regal Battery Park,Movie Theater,102 North End Ave.,NaN,40.715004,-74.014935,"[{'label': 'display', 'lat': 40.71500399534887, 'lng': -74.01493484218707}]",34,10282,US,New York,NY,United States,"[102 North End Ave., New York, NY 10282, United States]",NaN,49e7ce15f964a5201d651fe3
6,Shake Shack,Burger Joint,215 Murray St,btwn West St & North End Ave,40.715326,-74.014750,"[{'label': 'display', 'lat': 40.71532572032048, 'lng': -74.01475032559945}]",64,10282,US,New York,NY,United States,"[215 Murray St (btwn West St & North End Ave), New York, NY 10282, United States]",NaN,4dda977ad22d38ef4332c419
7,Davidoff of Geneva,Smoke Shop,230 Vesey St,Liberty St,40.713111,-74.015130,"[{'label': 'display', 'lat': 40.713111179157615, 'lng': -74.01512994530287}]",206,10281,US,New York,NY,United States,"[230 Vesey St (Liberty St), New York, NY 10281, United States]",NaN,56802dd1498ecedae4b17730
8,Conrad New York,Hotel,102 N End Ave,NaN,40.715030,-74.015414,"[{'label': 'display', 'lat': 40.71503, 'lng': -74.0154143}]",10,10282,US,New York,NY,United States,"[102 N End Ave, New York, NY 10282, United States]",Battery Park City,4f22ca77e4b0ed3396a83a05
9,North End Ave Dog Run,Dog Run,North End Ave,Warren St,40.716876,-74.016149,"[{'label': 'display', 'lat': 40.71687589247016, 'lng': -74.01614906976579}]",224,10282,US,New York,NY,United States,"[North End Ave (Warren St), New York, NY 10282, United States]",NaN,4bedbb23e24d20a116e77114


In [197]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [200]:
url='https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)
url

'https://api.foursquare.com/v2/venues/trending?client_id=RY5QQ3HUOBSHN0KS14RLJTVFKZAADYAQBV0R2BZ4OJ4V22O5&client_secret=TPZFUXLDONC0JPZNX2NZA0Z2VHFMF3IO0RTQDCMVG14FVTMB&ll=40.7149555,-74.0153365&v=20180604'

In [201]:
requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5e0eb52178a484001bf8b605'},
 'response': {'venues': []}}

In [206]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0eb5ba0be7b4001bc0fe7b'},
 'response': {'venues': []}}

In [205]:
latitude = 40.715337
longitude = -74.008848